## No longer needed for now
#### Parsing with Athena using:

`ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.OpenCSVSerde' WITH SERDEPROPERTIES ("separatorChar" = ",","quoteChar" = "\\""`

In [4]:
import boto3
import pandas as pd
import json
from io import StringIO

In [5]:
s3_bucket = "ads-508-final"
s3_key = "playstation/games/games.csv"

In [6]:
s3 = boto3.client("s3")

# Read the CSV file from S3
response = s3.get_object(Bucket=s3_bucket, Key=s3_key)
csv_data = response["Body"].read().decode("utf-8")

# Load CSV into a Pandas DataFrame
df = pd.read_csv(StringIO(csv_data))


columns_to_wrap = ['developers', 'publishers', 'genres', 'supported_languages']

for col in columns_to_wrap:
    df[col] = df[col].apply(lambda x: f'"{x}"' if isinstance(x, str) and x.startswith("[") and x.endswith("]") and not x.startswith('"') else x)

# Convert DataFrame to CSV in memory
csv_buffer = StringIO()
df.to_csv(csv_buffer, index=False, quotechar='"', encoding="utf-8")

# Upload back to S3 (overwrite the existing file)
s3.put_object(Bucket=s3_bucket, Key=s3_key, Body=csv_buffer.getvalue())

print(f"File successfully updated in s3://{s3_bucket}/{s3_key}")

File successfully updated in s3://ads-508-final/playstation/games/games.csv
